# Coursera IBM Data Science Capstone Week 3 

## Neighborhood Clustering in Toronto

### Catherine Gronemann


## Part 1: Scraping html from a wikipedia table, converting it into a df and transforming df

*In Terminal install the following packages:*  
**pip install beautifulsoup4** --> Beautiful Soup is a Python library for pulling data out of HTML and XML files  
**pip install lxml** --> an html parser  
**pip install request** --> request libary  

### Scraping the data

In [47]:
#Importing Libaries:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#page, to handle the contents of the website
page = requests.get(url)
#parse website and store contents under doc
doc = lh.fromstring(page.content)
#parse data that is stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows - should all be the same (= same number of columns)
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Parsing the first row as header:

In [48]:
#Create empty list
col=[]
i=0

#For each column, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

col

[('Postcode', []), ('Borough', []), ('Neighbourhood\n', [])]

### Parsing the data from row 2+ (since first row is the header):

In [49]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table --> insert correct number of columns here <--
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
#Check the length of each column - they should all be the same:
[len(C) for (title,C) in col]

[288, 288, 288]

### Create the DataFrame

In [50]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


### Transform the DataFrame to Course Specifications

In [51]:
#Rename Columns to coursera specification 
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [52]:
#delete '\n' from column Neighborhood strings
df['Neighborhood'] = df['Neighborhood'].astype(str).str[:-1]
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [53]:
#select only rows that do not contain "not assigned" in the borough column
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows 
will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table:

In [54]:
#Combining PostalCode and Borough to unique combinations and join neighborhoods of these groups together 
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will 
be Queen's Park.

In [55]:
#finding the concerned rows 
df_check = df[df['Neighborhood']=='Not assigned']
df_check

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [56]:
#Assigning the Borough's name of same row to concerned neighborhood
df.loc[85,'Neighborhood'] = df.loc[85,'Borough']
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [57]:
#shape of the resulting df
df.shape 

(103, 3)

## Part 2: Retrieving Geodata with geocoder

In order to get the latitude and the longitude of each Postcode a Geocoder is needed. The Google Maps Geocoding API is cost intensive, therefore the free Geocoder Python package is used instead: https://geocoder.readthedocs.io/index.html 

*In Terminal install the following packages:*
**pip install geocoder**

In [58]:
'''#import geocoder:
import geocoder 

# initialize variable to None
lat_lng_coords = None

# loop until the coordinates are fetched
#for index, row in df.iterrows():
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format('MB1')) #row[0]
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]'''

"#import geocoder:\nimport geocoder \n\n# initialize variable to None\nlat_lng_coords = None\n\n# loop until the coordinates are fetched\n#for index, row in df.iterrows():\nwhile(lat_lng_coords is None):\n    g = geocoder.google('{}, Toronto, Ontario'.format('MB1')) #row[0]\n    lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]"

Geocoder is not performing - therefore provided csv is used. 

In [59]:
#load csv of PostCodes latitudes and longitudes 
latlong_df = pd.read_csv("http://cocl.us/Geospatial_data")
latlong_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
#renaiming PostalCode to match the existing table for the merge
latlong_df.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlong_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:
#Merging the two dataframes on PostalCode
left = df
right = latlong_df

result_df = pd.merge(left, right, on='PostalCode')
result_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3: Clustering and Visualizing the data

Explore and cluster the neighborhoods in Toronto. 

Make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together

In [62]:
#importinng necessary libaries
import numpy as np 

#import geocoder
import geocoder 

# convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Using geopy library to get the latitude and longitude values of Toronto 

In [63]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Creating a map of Toronto with neighborhoods displayed on it

In [64]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result_df['Latitude'], result_df['Longitude'], result_df['Borough'], result_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Utilizing the Foursquare API to explore the Boroughs and segment them

**Folium** is a great visualization library. Its possible to zoom into the maps and click on each circle mark to reveal the name of the respective borough and neighborhood.

In [65]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'UWB1I3EJTNIXLJI0N4WRXDTSPQNHN43A1AW3CJ0LZYXSZ1UK' # your Foursquare ID
CLIENT_SECRET = 'X4EMDHRH4SVMAS5HRHBF2HJKYTZAPVMABEAHXDKRNCPLFBP3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UWB1I3EJTNIXLJI0N4WRXDTSPQNHN43A1AW3CJ0LZYXSZ1UK
CLIENT_SECRET:X4EMDHRH4SVMAS5HRHBF2HJKYTZAPVMABEAHXDKRNCPLFBP3


In [66]:
#library to handle JSON files
import json 

#tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

#### Explore Neighborhoods Venues and enrich the dataframe with it

In [67]:
#function to define nearby venues

#limit of number of venues returned by Foursquare API
LIMIT = 50 
#define radius for search
radius = 400 

def getNearbyVenues(names, latitudes, longitudes, radius):
    
    #create empty list
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### running the above function on each neighborhood and creating a new df called *toronto_venues*

In [68]:
toronto_venues = getNearbyVenues(names=result_df['Neighborhood'],
                                   latitudes=result_df['Latitude'],
                                   longitudes=result_df['Longitude'], 
                                   radius = 400 
                                  )
print(toronto_venues.shape)
toronto_venues.head()

(1271, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


### Merging new retrieved data with existing df info of Boroughs and PostCodes

In [69]:
#Merging the two dataframes Neighborhood
left = result_df
right = toronto_venues

borough_venues = pd.merge(left, right, on='Neighborhood')
borough_venues.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [70]:
# drop columns Latitude and Longitude since they are duplicates now
borough_venues = borough_venues.drop(columns=['Latitude', 'Longitude'])
borough_venues.head()

,PostalCode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


### Checking how many venues were returned for each Borough

In [71]:
borough_venues.groupby('Borough').count()

,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,,,
Central Toronto,58,58,58,58,58,58,58,58
Downtown Toronto,660,660,660,660,660,660,660,660
East Toronto,95,95,95,95,95,95,95,95
East York,52,52,52,52,52,52,52,52
Etobicoke,45,45,45,45,45,45,45,45
Mississauga,6,6,6,6,6,6,6,6
North York,177,177,177,177,177,177,177,177
Queen's Park,14,14,14,14,14,14,14,14
Scarborough,47,47,47,47,47,47,47,47


### Checking how many unique venue categories exist in order to decide if it makes sence to transfer there categorical values to numerical (one hot encoding)

In [72]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 224 uniques categories.


224 categories seems fine for onehot encoding, so the categorical data is changed to numerical with the goal of applying k-means clustering afterwards.  

### Changing categorical data of "venue category" to numerical for clustering

In [73]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column to new onehot df
borough_onehot['Borough'] = borough_venues['Borough'] 

# move Borough column to the first column
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]

borough_onehot.head()

,Borough,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
borough_onehot.shape

(1271, 225)

### Grouping rows by borough and taking the mean of occurrence frequency for each category

In [75]:
borough_grouped = borough_onehot.groupby('Borough').mean().reset_index()
borough_grouped

,Borough,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.017241,...,0.017241,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Downtown Toronto,0.00000,0.001515,0.001515,0.001515,0.001515,0.00303,0.001515,0.00303,0.016667,...,0.013636,0.001515,0.00000,0.004545,0.000000,0.004545,0.000000,0.001515,0.000000,0.001515
2,East Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.010526,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021053
3,East York,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.019231
4,Etobicoke,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000
5,Mississauga,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.166667,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.00565,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.005650,...,0.000000,0.005650,0.00565,0.005650,0.000000,0.005650,0.000000,0.005650,0.000000,0.000000
7,Queen's Park,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Scarborough,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.021277,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.009709,0.000000,0.00000,0.019417,0.000000,0.009709,0.009709,0.000000,0.000000,0.009709


### Creating a new dataframe and display the top 10 venues for each Borough

In [76]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = borough_grouped['Borough']

# fill df with most commmon venues
for ind in np.arange(borough_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Dessert Shop,Café,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Pharmacy
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Gastropub,Japanese Restaurant,Pizza Place,Deli / Bodega
2,East Toronto,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Park,Pizza Place,Café,Bookstore,Fast Food Restaurant,Gastropub
3,East York,Pizza Place,Sporting Goods Shop,Bank,Park,Liquor Store,Sandwich Place,Pharmacy,Indian Restaurant,Burger Joint,Coffee Shop
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Grocery Store,Park,Pharmacy,Bus Line,Beer Store,Fast Food Restaurant,Pool
5,Mississauga,Coffee Shop,Middle Eastern Restaurant,Hotel,American Restaurant,Burrito Place,Fried Chicken Joint,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Park,Restaurant,Pharmacy,Grocery Store,Japanese Restaurant,Italian Restaurant,Bank
7,Queen's Park,Coffee Shop,Sushi Restaurant,Italian Restaurant,Park,Persian Restaurant,College Cafeteria,Portuguese Restaurant,Café,Gym,American Restaurant
8,Scarborough,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Sandwich Place,Intersection,Chinese Restaurant,Bank,Pizza Place,Thai Restaurant,Skating Rink
9,West Toronto,Coffee Shop,Bar,Café,Pizza Place,Bakery,Asian Restaurant,Gym / Fitness Center,Men's Store,Restaurant,Music Venue


### Clustering the Boroughs into 3 clusters with *k*-means based on their venue categories

In [80]:
# set number of clusters
k = 3

borough_grouped_clustering = borough_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 2, 1, 1], dtype=int32)

### Creating a new dataframe that includes the clusters, the top 10 venues and the Geodata

In [91]:
# add clustering labels
# borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# getting latitude/longitude for each borough by adding mean of neighborhoods
borough_geodata = borough_venues.groupby('Borough')[['Neighborhood Latitude','Neighborhood Longitude']].mean()
#merging the cluster df with the geodata df
#borough_merged = borough_geodata.join(borough_venues_sorted.set_index('Borough'), on='Borough')

left = borough_venues_sorted
right = borough_geodata
borough_merged = pd.merge(left, right, on='Borough')

borough_merged.head()

,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood Latitude,Neighborhood Longitude
0,1,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Dessert Shop,Café,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Pharmacy,43.692569,-79.395972
1,1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Gastropub,Japanese Restaurant,Pizza Place,Deli / Bodega,43.652642,-79.382518
2,1,East Toronto,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Park,Pizza Place,Café,Bookstore,Fast Food Restaurant,Gastropub,43.668930,-79.336675
3,1,East York,Pizza Place,Sporting Goods Shop,Bank,Park,Liquor Store,Sandwich Place,Pharmacy,Indian Restaurant,Burger Joint,Coffee Shop,43.705811,-79.347312
4,1,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Grocery Store,Park,Pharmacy,Bus Line,Beer Store,Fast Food Restaurant,Pool,43.654335,-79.545978


### Visualizing the resulting clusters

In [92]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(borough_merged['Neighborhood Latitude'], borough_merged['Neighborhood Longitude'], borough_merged['Borough'], borough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters